In [1]:
# outdated
# spectrograms are stored as .npy files now
# filepaths are different
# some code may be missing

In [10]:
import numpy as np
import keras
from keras import callbacks
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from matplotlib import pyplot as plt
from IPython.display import clear_output
from sklearn.metrics import confusion_matrix
import gc

In [11]:
gc.enable()

In [3]:
img_row = 108
img_col = 108
path = "E:/database108/"
batch_size = 64
output_classes = 24

In [12]:
import random
import pandas as pd
x_train_ori = []
y_train_ori = []
x_test_ori = []
y_test_ori = []
for i in range(516):
    print(i)
    file = pd.read_csv(path + "database_{}.csv".format(i), dtype=str)
    array = file.values
    array = list(array)
    train_test = random.sample(range(100), 60)
    
    for j in train_test[:50]:
        sample = list(array[j])
        lst = []
        for x in sample[1:11665]:
            if type(x) == str:
                lst.append(float(x))
            elif type(x) == float:
                lst.append(x)
            else:
                print(type(x))
        x_train_ori.append(lst)
        y_train_ori.append(sample[0])
        
    for l in train_test[50:]:
        sample = list(array[l])
        lst = []
        for x in sample[1:11665]:
            if type(x) == str:
                lst.append(float(x))
            elif type(x) == float:
                lst.append(x)
            else:
                print(type(x))
        x_test_ori.append(lst)
        y_test_ori.append(sample[0])
    gc.collect()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [13]:
print(np.shape(x_train_ori))
print(np.shape(x_test_ori))
print(np.shape(y_train_ori))
print(np.shape(y_test_ori))

(25800, 11664)
(5160, 11664)
(25800,)
(5160,)


In [14]:
def y_convert(array, flag=0):
    y = []
    for i in array: # i is the name of each sample:
        name = i.split("_")
        name = name[1:5]
        if flag == 0:
            # category of appearance of a note
            category = np.zeros(24, dtype=int)
            for j in name:
                if j != "x":
                    num = int(j)
                    category[num-60] = 1
            y.append(list(category))
    return y

In [15]:
lst_1 = []
lst_2 = []

for i in y_train_ori:
    label = "".join(i.split())
    lst_1.append(label)
for j in y_test_ori:
    label = "".join(j.split())
    lst_2.append(label)
    
y_train_ori = y_convert(lst_1)
y_test_ori = y_convert(lst_2)

In [16]:
y_train_ori = np.array(y_train_ori)
y_test_ori = np.array(y_test_ori)
x_train_ori = np.array(x_train_ori)
x_test_ori = np.array(x_test_ori)

In [17]:
x_train_ori = x_train_ori.flatten()
x_train_ori = x_train_ori.reshape(25800, img_row, img_col, 1)
x_test_ori = x_test_ori.flatten()
x_test_ori = x_test_ori.reshape(5160, img_row, img_col, 1)

In [18]:
print(x_train_ori.shape)
input_shape = (108, 108, 1)

(25800, 108, 108, 1)


In [19]:
def build_model_E():
    early_stop = callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=3, verbose=1, mode="min")
    
    filepath = "Best-weights-my_model-{epoch:03d}-{loss:4f}-{acc:4f}.hdf5"
    checkpoint = callbacks.ModelCheckpoint(filepath, monitor="val_loss", verbose=1, save_best_only= True, mode="min")
    
    callbacks_list = [early_stop, checkpoint]
    
    #build the model
    model = Sequential()
    
    #first convolutional layer
    model.add(Conv2D(32, (3,3), input_shape=input_shape, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #second convolutional layer
    model.add(Conv2D(32, (3,3), activation="relu", border_mode="same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #third convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))

    #forth convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #fifth convulutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.3))
    
    #first FC layer
    model.add(Flatten())
    model.add(Dense(64, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    
    #second FC layer
    model.add(Dense(256, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    #output layer
    model.add(Dense(24, activation="sigmoid", kernel_initializer="uniform"))
    
    model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])
    return model

In [21]:
classifier_E = build_model_E()
hist_E = classifier_E.fit(x_train_ori, y_train_ori, batch_size=32, nb_epoch=200, verbose=1, 
                          validation_data=(x_test_ori, y_test_ori))

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_ep

Train on 25800 samples, validate on 5160 samples
Epoch 1/200
25800/25800 [==============================] - 25s 982us/step - loss: 0.0412 - acc: 0.2817 - val_loss: 0.0012 - val_acc: 0.1994
Epoch 2/200
10752/25800 [===========>..................] - ETA: 11s - loss: 0.0042 - acc: 0.2753

KeyboardInterrupt: 